***PARTE II***

A continuación se repiten los pasos realizados en la Parte I utilizando el modelo MLP

Nuevamente se cargan las librerías a utilizar en esta sección.

#Librerías


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from string import punctuation
from termcolor import colored
from collections import Counter
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#Cargar datos de Kaggle

Luego se prosigue cargando los datos desde kaggle

In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amaliaoxandaberro","key":"dc3caded742346409c8426f0109c18a1"}'}

In [ ]:
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c sesgos-en-el-dataset-de-snli

  0% 0.00/200k [00:00<?, ?B/s]
100% 200k/200k [00:00<00:00, 61.6MB/s]
 98% 25.0M/25.6M [00:00<00:00, 69.6MB/s]
100% 25.6M/25.6M [00:00<00:00, 85.1MB/s]
  0% 0.00/152k [00:00<?, ?B/s]
100% 152k/152k [00:00<00:00, 46.0MB/s]
  0% 0.00/440k [00:00<?, ?B/s]
100% 440k/440k [00:00<00:00, 133MB/s]


In [ ]:
!unzip test_data.hdf5.zip
!unzip train_data.hdf5.zip
!unzip valid_data.hdf5.zip

Archive:  test_data.hdf5.zip
  inflating: test_data.hdf5          
Archive:  train_data.hdf5.zip
  inflating: train_data.hdf5         
Archive:  valid_data.hdf5.zip
  inflating: valid_data.hdf5         


In [ ]:
df_train = pd.read_hdf("/content/train_data.hdf5")
df_valid = pd.read_hdf("/content/valid_data.hdf5")
df_test = pd.read_hdf("/content/test_data.hdf5")

In [ ]:
text_train = df_train["text"].tolist()
labels_train = df_train["gold_label"].tolist()

text_valid = df_valid["text"].tolist()
labels_valid = df_valid["gold_label"].tolist()

text_test = df_test["text"].tolist()

#CountVectorizer

A continuación se utiliza el vectorizador Count Vectorizer para luego crear el modelo de MLP. Buscando llegar al mejor resultado se realizaron varios modelos modificando cada vez las distintas variables. En el caso del vectorizador, se cambiaron los parámetros min_df, max_df y ngram_range, pero también se cambiaron la cantidad de capas, la cantidad de neuronas por capa, la función de activación, batch size, epochs, learning rate y optimizador utilizado. 

In [ ]:
cv = CountVectorizer(min_df=100, max_df = 0.8, ngram_range = (1,1))
cv_train = cv.fit_transform(text_train)
cv_valid = cv.transform(text_valid)

#Modelo MLP


In [ ]:
model = Sequential()
model.add(Dense(5, input_shape = (cv_train.shape[1],), activation='relu'))
model.add(Dense(5, input_shape = (cv_train.shape[1],), activation='relu'))
model.add(Dense(3, activation="softmax", input_shape = (cv_train.shape[1],)))
opt = SGD(learning_rate=0.1)

model.compile(loss = "categorical_crossentropy", metrics=["accuracy"], optimizer= opt)

In [ ]:
ohe = OneHotEncoder()
ohe.fit(np.array(labels_train).reshape(-1,1))
labels_train_enc = ohe.transform(np.array(labels_train).reshape(-1,1))
labels_val_enc = ohe.transform(np.array(labels_valid).reshape(-1,1))

In [ ]:
#callback = EarlyStopping(monitor='val_accuracy', patience=7, min_delta=0.000001)

model.fit(x=cv_train.toarray(), y = labels_train_enc.toarray(), batch_size = 256, validation_data = (cv_valid.toarray(), labels_val_enc.toarray()),  verbose=1, epochs=100)

Epoch 1/100
2146/2146 [==============================] - 16s 7ms/step - loss: 0.9633 - accuracy: 0.5593 - val_loss: 0.8694 - val_accuracy: 0.6116
Epoch 2/100
2146/2146 [==============================] - 15s 7ms/step - loss: 0.8606 - accuracy: 0.6145 - val_loss: 0.8456 - val_accuracy: 0.6240
Epoch 3/100
2146/2146 [==============================] - 15s 7ms/step - loss: 0.8493 - accuracy: 0.6206 - val_loss: 0.8420 - val_accuracy: 0.6223
Epoch 4/100
2146/2146 [==============================] - 15s 7ms/step - loss: 0.8462 - accuracy: 0.6218 - val_loss: 0.8407 - val_accuracy: 0.6250
Epoch 5/100
2146/2146 [==============================] - 15s 7ms/step - loss: 0.8450 - accuracy: 0.6221 - val_loss: 0.8399 - val_accuracy: 0.6251
Epoch 6/100
2146/2146 [==============================] - 15s 7ms/step - loss: 0.8445 - accuracy: 0.6221 - val_loss: 0.8394 - val_accuracy: 0.6250
Epoch 7/100
2146/2146 [==============================] - 15s 7ms/step - loss: 0.8441 - accuracy: 0.6220 - val_loss: 0.8398 -

#Resultados


In [ ]:
prediction = model.predict(cv_valid)
labels_val = ohe.inverse_transform(np.array(prediction))

In [ ]:
print("Recall: "+ str(recall_score(labels_valid, labels_val, average='macro')))
print("Precision: "+ str(precision_score(labels_valid, labels_val, average='macro')))
print("F1: "+ str(f1_score(labels_valid, labels_val, average='macro')))
print("Accuracy: "+ str(accuracy_score(labels_valid, labels_val)))

Recall: 0.6326381215819751
Precision: 0.6327334644511301
F1: 0.632565416358763
Accuracy: 0.6326966063808169


Los resultados obtenidos cambiando los distintos parámetros se encuentran en un planilla adjunta al trabajo.

#Conclusiones




*   Para el modelo NB el preprocesamiento de tipo BOW (bag of words), que ignora la estructura de las frases y solo tiene en cuenta el significado de cada palabra individualmente y la cantidad de veces que aparece, no mejora el modelo. Esto puede indicar que el sesgo se encuentra en la estructura y no en el sentido de las palabras. 
*   Para el modelo NB, se obtuvieron mejores resultados con el vectorizador TFIDF que con Count Vectorizer.
*   Al evaluar el modelo con los datos de train, se obtiene un accuracy más alto que con los datos de validación. No se alcanza una diferencia considerable como para detectar presencia de overfitting.
*   Dado que el accuracy es razonablemente alto (entre 60% - 65%), se concluye que hay sesgos en el dataset (localizados en la estructura de los datos).
*   En el modelo de NB el mejor resultado de accuracy que se obtuvo fue de 65,5761024182076, mientras que en el modelo de MLP se obtuvo un valor de 63,6151188782767. Los parámetros con los que se obtuvieron estos resultados se encuentran resaltados en las planillas adjuntas. 
*   Se esperaba un mejor resultado utilizando el modelo de MLP frente al de NB, ya que permite una mayor variación de parámetros y es un modelo mas complejo. Sin embargo, según los resultados obtenidos empíricamente hasta el momento esto no sucedió.






